In [1]:
import numpy as np
import pandas as pd
from itertools import product
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.quantum_info import Statevector
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import qiskit.tools.jupyter


In [2]:
def testCircuit(circ, shots, printCnt=False, plotCnt=False):
    backend = AerSimulator()
    qc_compiled = transpile(circ, backend)
    job_sim = backend.run(qc_compiled, shots=shots)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(circ)
    if printCnt:
        print(f"Counts: {counts}")
    if plotCnt:
        plot_histogram(counts)
    return counts

In [3]:
def result_to_df(result):
    df = pd.DataFrame(result)
    df = df.fillna(0).astype(int)
    return df

In [4]:
def result_to_truth_table(qIn, qOut, result):
    tt = []
    for x in result:
        keyList = list(x.keys())
        for k in keyList:
            tmp = [*k]
            tmp = [int(i) for i in tmp]
            tmp.reverse()
            tt.append(tmp)
    tt.sort()
    df = pd.DataFrame(tt)
    colName = ['qIn'] * qIn + ['qOut'] * qOut
    df.columns = colName
    return df

In [5]:
def apply_circ(gate='AND', printResult=False, returnData=False):
    if gate in ['AND', 'OR', 'XOR', 'NAND', 'NOR', 'XNOR']:
        qIn = 2
        qOut = 1
    elif gate in ['Full-Adder', 'Half-Adder']:
        qIn = 3
        qOut = 2
    q = qIn + qOut
    c = q
    result = []
    # for i in range(repeat):
    for bits in product([0, 1], repeat=qIn):
        circ = QuantumCircuit(q, c)
        # Initialize qubits state
        for i, bit in enumerate(bits):
            if bit == 1:
                circ.x(i)
        circ.barrier(range(q))
        # gate
        if gate == 'AND':
            circ.ccx(0, 1, 2)
        elif gate == 'OR':
            circ.cx(0, 2)
            circ.cx(1, 2)
            circ.ccx(0, 1, 2)
        elif gate == 'XOR':
            circ.cx(0, 2)
            circ.cx(1, 2)
        elif gate == 'NAND':
            circ.ccx(0, 1, 2)
            circ.x(2)
        elif gate == 'NOR':
            circ.cx(0, 2)
            circ.cx(1, 2)
            circ.ccx(0, 1, 2)
            circ.x(2)
        elif gate == 'XNOR':
            circ.cx(0, 2)
            circ.cx(1, 2)
            circ.x(2)
        elif gate == 'Full-Adder':
            circ.cx(0, 3)
            circ.cx(1, 3)
            circ.cx(2, 3)
            circ.ccx(0, 1, 4)
            circ.ccx(0, 2, 4)
            circ.ccx(1, 2, 4)
        # END gate
        circ.barrier(range(q))
        circ.measure(range(q), range(q))
        # print(testCircuit(circ, 1024, False, False))
        result.append(testCircuit(circ, 1024, False, False))

    distribution = result_to_df(result)
    truth_table = result_to_truth_table(qIn, qOut, result)
    
    if printResult:
        print(f"Result: {result}")
        print(f"Distribution: \n{distribution}")
        print(f"Truth Table: \n{truth_table}")

    if returnData:
        return result, distribution, truth_table

## AND gate

In [6]:
apply_circ('AND', True)

Result: [{'000': 1024}, {'010': 1024}, {'001': 1024}, {'111': 1024}]
Distribution: 
    000   010   001   111
0  1024     0     0     0
1     0  1024     0     0
2     0     0  1024     0
3     0     0     0  1024
Truth Table: 
   qIn  qIn  qOut
0    0    0     0
1    0    1     0
2    1    0     0
3    1    1     1


## OR gate

In [7]:
apply_circ('OR', True)


Result: [{'000': 1024}, {'110': 1024}, {'101': 1024}, {'111': 1024}]
Distribution: 
    000   110   101   111
0  1024     0     0     0
1     0  1024     0     0
2     0     0  1024     0
3     0     0     0  1024
Truth Table: 
   qIn  qIn  qOut
0    0    0     0
1    0    1     1
2    1    0     1
3    1    1     1


## XOR gate

In [8]:
apply_circ('XOR', True)

Result: [{'000': 1024}, {'110': 1024}, {'101': 1024}, {'011': 1024}]
Distribution: 
    000   110   101   011
0  1024     0     0     0
1     0  1024     0     0
2     0     0  1024     0
3     0     0     0  1024
Truth Table: 
   qIn  qIn  qOut
0    0    0     0
1    0    1     1
2    1    0     1
3    1    1     0


## NAND gate

In [9]:
apply_circ('NAND', True)

Result: [{'100': 1024}, {'110': 1024}, {'101': 1024}, {'011': 1024}]
Distribution: 
    100   110   101   011
0  1024     0     0     0
1     0  1024     0     0
2     0     0  1024     0
3     0     0     0  1024
Truth Table: 
   qIn  qIn  qOut
0    0    0     1
1    0    1     1
2    1    0     1
3    1    1     0


## NOR gate

In [10]:
apply_circ('NOR', True)

Result: [{'100': 1024}, {'010': 1024}, {'001': 1024}, {'011': 1024}]
Distribution: 
    100   010   001   011
0  1024     0     0     0
1     0  1024     0     0
2     0     0  1024     0
3     0     0     0  1024
Truth Table: 
   qIn  qIn  qOut
0    0    0     1
1    0    1     0
2    1    0     0
3    1    1     0


## XNOR gate

In [11]:
apply_circ('XNOR', True)

Result: [{'100': 1024}, {'010': 1024}, {'001': 1024}, {'111': 1024}]
Distribution: 
    100   010   001   111
0  1024     0     0     0
1     0  1024     0     0
2     0     0  1024     0
3     0     0     0  1024
Truth Table: 
   qIn  qIn  qOut
0    0    0     1
1    0    1     0
2    1    0     0
3    1    1     1


## Full-Adder Circuit

In [12]:
apply_circ('Full-Adder', True)

Result: [{'00000': 1024}, {'01100': 1024}, {'01010': 1024}, {'10110': 1024}, {'01001': 1024}, {'10101': 1024}, {'10011': 1024}, {'11111': 1024}]
Distribution: 
   00000  01100  01010  10110  01001  10101  10011  11111
0   1024      0      0      0      0      0      0      0
1      0   1024      0      0      0      0      0      0
2      0      0   1024      0      0      0      0      0
3      0      0      0   1024      0      0      0      0
4      0      0      0      0   1024      0      0      0
5      0      0      0      0      0   1024      0      0
6      0      0      0      0      0      0   1024      0
7      0      0      0      0      0      0      0   1024
Truth Table: 
   qIn  qIn  qIn  qOut  qOut
0    0    0    0     0     0
1    0    0    1     1     0
2    0    1    0     1     0
3    0    1    1     0     1
4    1    0    0     1     0
5    1    0    1     0     1
6    1    1    0     0     1
7    1    1    1     1     1
